In [1]:
#this is copied version https://towardsdatascience.com/automate-stacking-in-python-fc3e7834772e

In [2]:
#To get started, let’s first import the packages we are going to need for our project:

In [30]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier 
from xgboost import XGBClassifier
from vecstack import stacking

In [2]:
"""Now we are ready to import our data and take a look at it to gain a better understanding o
f what it looks like:"""


'Now we are ready to import our data and take a look at it to gain a better understanding o\nf what it looks like:'

In [25]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

df = pd.read_csv(r"D:\python_code\data\otto_challenge\train.csv")
df.sample(5)
df.drop("id",axis=1)
#axis 默认为0，指删除index，因此删除columns时要指定axis=1；

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1
5,2,1,0,0,7,0,0,0,0,0,...,0,3,0,0,0,0,2,0,0,Class_1
6,2,0,0,0,0,0,0,2,0,1,...,1,1,0,0,0,0,0,0,1,Class_1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Class_1
8,0,0,0,0,0,0,0,4,0,0,...,0,2,0,0,0,0,0,0,1,Class_1
9,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,Class_1


In [26]:
df.describe

<bound method NDFrame.describe of           id  feat_1  feat_2  feat_3  feat_4  feat_5  feat_6  feat_7  feat_8  \
0          1       1       0       0       0       0       0       0       0   
1          2       0       0       0       0       0       0       0       1   
2          3       0       0       0       0       0       0       0       1   
3          4       1       0       0       1       6       1       5       0   
4          5       0       0       0       0       0       0       0       0   
5          6       2       1       0       0       7       0       0       0   
6          7       2       0       0       0       0       0       0       2   
7          8       0       0       0       0       0       0       0       0   
8          9       0       0       0       0       0       0       0       4   
9         10       0       0       0       0       0       0       1       0   
10        11       0       1       1       2       0       0       2       1   
11    

In [27]:
"""Following this, we are going to separate the response from the input variables and perform 
an 80:20 train-test-split following the example on vecstacks’ documentation."""

'Following this, we are going to separate the response from the input variables and perform \nan 80:20 train-test-split following the example on vecstacks’ documentation.'

In [46]:
y = df.target.values
y = LabelEncoder().fit_transform(y)
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=36)

In [65]:
y[-200:]

array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8])

In [66]:
y.shape

(61878,)

In [67]:
y_test[-200:]

array([1, 6, 1, 5, 5, 1, 1, 2, 1, 5, 7, 7, 5, 2, 1, 1, 5, 5, 1, 8, 1, 2,
       7, 1, 1, 4, 0, 1, 2, 1, 0, 7, 5, 0, 1, 5, 1, 8, 1, 5, 2, 7, 5, 7,
       5, 4, 7, 5, 1, 1, 2, 5, 8, 5, 5, 8, 5, 8, 2, 7, 2, 3, 8, 5, 8, 5,
       3, 3, 1, 1, 5, 4, 5, 1, 1, 1, 2, 1, 5, 5, 1, 7, 2, 7, 7, 5, 4, 4,
       2, 1, 3, 3, 2, 1, 5, 5, 7, 3, 7, 5, 5, 5, 1, 5, 7, 6, 5, 7, 2, 7,
       1, 4, 8, 2, 4, 2, 1, 7, 2, 3, 1, 8, 5, 7, 7, 1, 7, 1, 5, 8, 5, 7,
       2, 7, 7, 5, 6, 2, 2, 8, 7, 2, 7, 1, 8, 5, 6, 7, 2, 2, 7, 5, 8, 1,
       1, 2, 5, 1, 6, 1, 1, 1, 5, 7, 1, 4, 1, 6, 1, 2, 5, 7, 7, 5, 4, 7,
       5, 1, 1, 5, 7, 2, 5, 0, 7, 1, 1, 5, 5, 5, 1, 7, 7, 6, 1, 2, 2, 2,
       5, 1])

In [57]:
#It is time now to define a few first level models for our stacked generalization. 

In [49]:
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3)
]

In [50]:
"""Taking the next part of code from the documentation, we are essentially performing 
the GIF’s first part using first level models to make predictions:"""

'Taking the next part of code from the documentation, we are essentially performing \nthe GIF’s first part using first level models to make predictions:'

In [51]:
S_train, S_test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [9]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.99927290]
    fold  1:  [0.99975761]
    fold  2:  [0.99967679]
    fold  3:  [0.99991917]
    ----
    MEAN:     [0.99965662] + [0.00023810]
    FULL:     [0.99965658]

model  1:     [RandomForestClassifier]
    fold  0:  [0.58660527]
    fold  1:  [0.59376262]
    fold  2:  [0.58896251]
    fold  3:  [0.59833482]
    ----
    MEAN:     [0.59191630] + [0.00451493]
    FULL:     [0.59191548]

model  2:     [XGBClassifier]
    fold  0:  [0.99975763]
    fold  1:  [0.99975761]
    fold  2:  [0.99991920]
    fold  3:  [0.99991917]
    ----
    MEAN:     [0.99983840] + [0.00008078]
    FULL:     [0.99983839]



In [52]:
"""Again, referring to the GIF, all that’s left to do now is fit the second level model(s) 
of our choice on our predictions to make our final predictions. In our case, we are going to 
use an XGBoost Classifier. This step is not significantly different from a regular 
fit-and-predict in sklearn except for the fact that instead of using X_train to train 
our model, we are using our predictions S_train."""

'Again, referring to the GIF, all that’s left to do now is fit the second level model(s) \nof our choice on our predictions to make our final predictions. In our case, we are going to \nuse an XGBoost Classifier. This step is not significantly different from a regular \nfit-and-predict in sklearn except for the fact that instead of using X_train to train \nour model, we are using our predictions S_train.'

In [53]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))


Final prediction score: [0.99991920]
